<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.1 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=6be5162b51b3c462e7e6d540380041392acdb83de6f6767d3fcc6796318e2320
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


In [27]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [28]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [29]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [30]:
mypf['InPortfolio'].value_counts()

0    55
1     9
Name: InPortfolio, dtype: int64

In [31]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [32]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df.ta.macd(close='Close Price', fast=12, slow=26, append=True)
    stock_df['Trend'] = 'Down'
    stock_df.loc[stock_df['MACD_12_26_9'] > stock_df['MACDs_12_26_9'], 'Trend'] = 'Up'
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJAJFINSV.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BANKBARODA.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[************

In [33]:
stock_prec_dev('^NSEI').tail(5)

[*********************100%***********************]  1 of 1 completed


,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend
Date,,,,,,,,,,,,,
2022-12-20,18385.0,^NSEI,5.23,0.62,0.29,18812.0,15294.0,87.87,12.13,64.313904,-60.157330,124.471234,Down
2022-12-21,18199.0,^NSEI,4.13,0.59,0.18,18812.0,15294.0,82.58,17.42,37.352647,-69.694869,107.047516,Down
2022-12-22,18127.0,^NSEI,3.68,0.55,0.11,18812.0,15294.0,80.54,19.46,10.079837,-77.574143,87.653980,Down
2022-12-23,17807.0,^NSEI,1.82,0.51,0.01,18812.0,15294.0,71.43,28.57,-36.973466,-99.701957,62.728491,Down
2022-12-26,18015.0,^NSEI,2.98,0.47,-0.08,18812.0,15294.0,77.33,22.67,-56.840694,-95.655348,38.814654,Down


In [34]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [35]:
# portfolio condition
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 9


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
1,2022-12-26,446.0,ICICIPRULI.NS,-13.42,-1.36,-1.46,616.0,440.0,3.35,96.65,-9.736233,0.317136,-10.053368,Up,640.0,43.50
2,2022-12-26,686.0,MOTILALOFS.NS,-9.95,-0.92,0.14,971.0,654.0,10.14,89.86,4.256676,2.140263,2.116413,Up,930.0,35.57
3,2022-12-26,1243.0,SBILIFE.NS,2.25,0.33,-0.40,1332.0,1025.0,71.11,28.89,-5.097989,-4.505698,-0.592291,Down,1607.0,29.28
12,2022-12-26,3898.0,NAUKRI.NS,-7.16,-0.24,0.07,5696.0,3375.0,22.54,77.46,10.412310,-25.616550,36.028860,Down,4677.0,19.98
13,2022-12-26,569.0,HDFCLIFE.NS,0.58,0.35,0.35,674.0,504.0,38.41,61.59,3.827533,-3.112877,6.940411,Down,681.0,19.68
22,2022-12-26,1064.0,MUTHOOTFIN.NS,-6.46,-0.44,-0.09,1540.0,956.0,18.58,81.42,-3.196781,-6.696933,3.500152,Down,1217.0,14.38
24,2022-12-26,1502.0,INFY.NS,-4.61,-0.32,-0.58,1940.0,1365.0,23.90,76.10,-19.210880,-11.236323,-7.974557,Down,1715.0,14.18
26,2022-12-26,1001.0,TECHM.NS,-13.93,-1.29,-0.86,1806.0,965.0,4.28,95.72,-15.131044,-6.493058,-8.637986,Down,1139.0,13.79
54,2022-12-26,3429.0,DIVISLAB.NS,-8.20,-0.91,0.12,4678.0,3266.0,11.53,88.47,11.581818,21.141044,-9.559226,Up,3560.0,3.82


In [36]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 3


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
7,2022-12-26,521.0,JUBLFOOD.NS,-10.28,-1.03,-1.27,786.0,464.0,17.80,82.20,-11.665284,0.558442,-12.223726,Up,643.0,23.42
14,2022-12-26,2481.0,TITAN.NS,-0.28,0.12,-0.81,2770.0,1936.0,65.36,34.64,-39.536156,-8.089343,-31.446813,Down,2967.0,19.59
15,2022-12-26,1111.0,HAVELLS.NS,-10.87,-0.88,-1.32,1406.0,1070.0,12.07,87.93,-28.489393,-6.613333,-21.876061,Down,1328.0,19.53


In [37]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     22.401895
2     18.318055
3     15.078793
12    10.289422
13    10.134926
22     7.405500
24     7.302503
26     7.101658
54     1.967247
Name: Upside%, dtype: float64

In [38]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 9


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
0,2022-12-26,434.0,GODREJIND.NS,-8.00,-0.64,-0.45,644.0,405.0,12.24,87.76,-0.045375,-5.757325,5.711951,Down,638.0,47.00
4,2022-12-26,385.0,TATAMOTORS.NS,-11.13,-0.69,-1.20,525.0,372.0,8.37,91.63,-8.599497,-4.472239,-4.127258,Down,495.0,28.57
5,2022-12-26,239.0,ZEEL.NS,-8.41,-0.38,-1.17,328.0,209.0,25.25,74.75,-5.201702,-2.478187,-2.723515,Down,303.0,26.78
6,2022-12-26,6434.0,BAJFINANCE.NS,-5.58,-0.38,-0.97,7853.0,5284.0,44.75,55.25,-108.194446,-5.076687,-103.117759,Down,8015.0,24.57
7,2022-12-26,521.0,JUBLFOOD.NS,-10.28,-1.03,-1.27,786.0,464.0,17.80,82.20,-11.665284,0.558442,-12.223726,Up,643.0,23.42
8,2022-12-26,3161.0,EICHERMOT.NS,0.17,0.38,-1.21,3850.0,2256.0,56.79,43.21,-72.922555,-12.110676,-60.811879,Down,3884.0,22.87
9,2022-12-26,893.0,ICICIBANK.NS,6.21,0.83,-0.44,953.0,654.0,80.00,20.00,-7.671116,-5.164384,-2.506732,Down,1094.0,22.51
10,2022-12-26,78.0,IBREALEST.NS,-12.36,-1.12,0.00,174.0,60.0,15.79,84.21,-0.843497,-1.273324,0.429828,Down,95.0,21.79
11,2022-12-26,74.0,NATIONALUM.NS,-10.78,-1.20,0.00,130.0,67.0,11.19,88.81,-0.036912,-0.691660,0.654748,Down,90.0,21.62


In [39]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 3


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
4,2022-12-26,385.0,TATAMOTORS.NS,-11.13,-0.69,-1.20,525.0,372.0,8.37,91.63,-8.599497,-4.472239,-4.127258,Down,495.0,28.57
23,2022-12-26,203.0,TATAPOWER.NS,-9.71,-0.89,-1.36,290.0,195.0,8.58,91.42,-4.799590,-2.199557,-2.600033,Down,232.0,14.29
33,2022-12-26,380.0,WIPRO.NS,-15.82,-1.55,-0.50,722.0,375.0,1.59,98.41,-4.031814,-2.487165,-1.544649,Down,425.0,11.84


In [40]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Premium%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Dev%_200', ascending=True)

qualified stocks: 3


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
34,2022-12-26,1629.0,HDFCBANK.NS,9.51,1.01,0.57,1662.0,1281.0,91.46,8.54,12.995141,-6.641704,19.636845,Down,1813.0,11.30
25,2022-12-26,929.0,AXISBANK.NS,17.71,1.90,1.13,950.0,626.0,93.43,6.57,13.760267,-4.275586,18.035853,Down,1060.0,14.10
59,2022-12-26,289.0,INDIANB.NS,39.71,3.86,1.89,300.0,132.0,93.57,6.43,5.991417,-2.603277,8.594694,Down,284.0,-1.73


In [41]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
53,2022-12-26,3253.0,TCS.NS,-2.32,-0.21,-0.18,4019.0,2982.0,26.12,73.88,-17.081238,-15.168048,-1.913190,Down,3385.0,4.06
44,2022-12-26,1032.0,HCLTECH.NS,-1.74,-0.10,-0.29,1352.0,883.0,31.71,68.29,-12.910351,-5.858183,-7.052168,Down,1117.0,8.24
41,2022-12-26,3056.0,ASIANPAINT.NS,-3.47,-0.22,-0.54,3576.0,2580.0,47.80,52.20,-22.304822,-10.679258,-11.625564,Down,3361.0,9.98
48,2022-12-26,4037.0,DMART.NS,-2.73,-0.24,-0.59,4731.0,3231.0,53.71,46.29,-24.371384,-6.931724,-17.439659,Down,4282.0,6.07
6,2022-12-26,6434.0,BAJFINANCE.NS,-5.58,-0.38,-0.97,7853.0,5284.0,44.75,55.25,-108.194446,-5.076687,-103.117759,Down,8015.0,24.57
28,2022-12-26,8257.0,MARUTI.NS,-2.78,-0.05,-1.10,9528.0,6769.0,53.93,46.07,-173.229479,-39.285125,-133.944354,Down,9356.0,13.31
5,2022-12-26,239.0,ZEEL.NS,-8.41,-0.38,-1.17,328.0,209.0,25.25,74.75,-5.201702,-2.478187,-2.723515,Down,303.0,26.78
21,2022-12-26,1951.0,DEEPAKNTR.NS,-8.04,-0.24,-1.50,2660.0,1720.0,24.61,75.39,-54.314388,-28.653363,-25.661025,Down,2260.0,15.84


In [42]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 0


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%


In [43]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
36,2022-12-26,1958.0,INDIGO.NS,4.16,0.69,0.84,2265.0,1549.0,57.16,42.84,30.739457,-12.505669,43.245127,Down,2176.0,11.13


In [44]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 6


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,Trend,Forecast,Upside%
10,2022-12-26,78.0,IBREALEST.NS,-12.36,-1.12,0.00,174.0,60.0,15.79,84.21,-0.843497,-1.273324,0.429828,Down,95.0,21.79
11,2022-12-26,74.0,NATIONALUM.NS,-10.78,-1.20,0.00,130.0,67.0,11.19,88.81,-0.036912,-0.691660,0.654748,Down,90.0,21.62
53,2022-12-26,3253.0,TCS.NS,-2.32,-0.21,-0.18,4019.0,2982.0,26.12,73.88,-17.081238,-15.168048,-1.913190,Down,3385.0,4.06
0,2022-12-26,434.0,GODREJIND.NS,-8.00,-0.64,-0.45,644.0,405.0,12.24,87.76,-0.045375,-5.757325,5.711951,Down,638.0,47.00
33,2022-12-26,380.0,WIPRO.NS,-15.82,-1.55,-0.50,722.0,375.0,1.59,98.41,-4.031814,-2.487165,-1.544649,Down,425.0,11.84
47,2022-12-26,4349.0,LTIM.NS,-12.25,-1.27,-1.26,7562.0,3824.0,14.04,85.96,-127.146184,-13.211929,-113.934255,Down,4654.0,7.01
